In [1]:
from google.colab import files

uploaded = files.upload()

Saving non_promoter.csv to non_promoter.csv
Saving promoter.csv to promoter.csv


In [4]:
import pandas as pd

# Load CSVs
promoters = pd.read_csv("promoter.csv")
non_promoters = pd.read_csv("non_promoter.csv")

# Add labels
promoters['label'] = 1
non_promoters['label'] = 0

# Combine both into one DataFrame
df = pd.concat([promoters, non_promoters], ignore_index=True)

In [6]:
# Basic info
print("=== Data Types ===")
print(df.dtypes)

print("\n=== Missing Values ===")
print(df.isnull().sum())

# Quick stats
df.describe(include='all')

=== Data Types ===
Promoter sequences        object
label                      int64
non Promoter sequences    object
dtype: object

=== Missing Values ===
Promoter sequences        30000
label                         0
non Promoter sequences    30000
dtype: int64


,Promoter sequences,label,non Promoter sequences
count,30000,60000.000000,30000
unique,29934,NaN,29995
top,TCCCCTTGGCGAGGGTTTGGCATGAAATCAATGGCCAGCAAGCTTG...,NaN,CACGCTATGCGTGATAATTTTTTTGTTTAATGAGCAGATATGAAGA...
freq,4,NaN,2
mean,NaN,0.500000,NaN
std,NaN,0.500004,NaN
min,NaN,0.000000,NaN
25%,NaN,0.000000,NaN
50%,NaN,0.500000,NaN
75%,NaN,1.000000,NaN


In [7]:
import pandas as pd
import numpy as np
from collections import Counter

In [11]:
df = df.rename(columns={
    'Promoter sequences': 'promoter_seq',
    'non Promoter sequences': 'non_promoter_seq'
})

In [22]:
# Combine sequences into one column
# Create a new column 'sequence' that contains either promoter or non-promoter sequences
df['sequence'] = np.where(
    df['label'] == 1,
    df['promoter_seq'],
    df['non_promoter_seq']
)

KeyError: 'promoter_seq'

In [21]:
# . Drop the original separate sequence columns
df = df.drop(columns=['promoter_seq', 'non_promoter_seq'])

KeyError: "['promoter_seq', 'non_promoter_seq'] not found in axis"

In [20]:
# . Check for missing values
print("\nMissing values after combining:")
print(df.isnull().sum())


Missing values after combining:
label       0
sequence    0
dtype: int64


In [18]:
 #. Remove rows with missing sequences
df = df.dropna(subset=['sequence'])

In [23]:
# . Validate DNA characters
valid_bases = set('ATCGN')
def validate_sequence(seq):
    return all(base in valid_bases for base in seq.upper())

valid_seqs = df['sequence'].apply(validate_sequence)
print(f"\nInvalid sequences found: {len(df) - sum(valid_seqs)}")


Invalid sequences found: 0


In [24]:
# Clean sequences (convert to uppercase, replace invalid with N)
df['sequence'] = df['sequence'].str.upper().apply(
    lambda s: ''.join(c if c in valid_bases else 'N' for c in s))


In [25]:
# Add sequence length
df['seq_length'] = df['sequence'].str.len()
print("\nSequence length distribution:")
print(df['seq_length'].describe())


Sequence length distribution:
count    60000.000000
mean       276.000000
std         25.000208
min        251.000000
25%        251.000000
50%        276.000000
75%        301.000000
max        301.000000
Name: seq_length, dtype: float64


In [26]:
# 10. Check class balance
print("\nClass balance:")
print(df['label'].value_counts(normalize=True))



Class balance:
label
1    0.5
0    0.5
Name: proportion, dtype: float64


In [27]:
#Save cleaned data
output_filename = "cleaned_promoter_data.csv"
df.to_csv(output_filename, index=False)

#Download from Colab
files.download(output_filename)

print(f"\nCleaned data saved and downloaded as '{output_filename}'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Cleaned data saved and downloaded as 'cleaned_promoter_data.csv'
